In [ ]:
#install required packages
#!pip3 install --upgrade pip setuptools wheel
#!pip3 install scikit-learn scipy keras pandas tensorflow numpy scikeras

In [ ]:
#import required libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import scipy 
import keras
import pandas as pd
import tensorflow as tf
import numpy as np
import scikeras
from scikeras.wrappers import KerasClassifier, KerasRegressor

In [ ]:
#Load data
train_data = pd.read_csv("~/ML blocking collisions/blocking_dataset/train_data_regr20.csv", header = None)
test_data = pd.read_csv("~/ML blocking collisions/blocking_dataset/test_data_regr20.csv", header = None)
# Shuffle data
train_data = train_data.sample(frac=1).reset_index(drop=True)
test_data = test_data.sample(frac=1).reset_index(drop=True)
#Select features and labels
train_features = train_data.iloc[:,1:]
train_labels = train_data.iloc[:,0]
test_features = test_data.iloc[:,1:]
test_labels = test_data.iloc[:,0]
train_features_full = np.array(train_features)
train_labels_full = np.array(train_labels)
test_features_full = np.array(test_features)
test_labels_full = np.array(test_labels)

In [ ]:
#split the data
X_train, X_valid, y_train, y_valid = train_test_split(train_features_full, train_labels_full)
X_test = test_features_full
y_test = test_labels_full

In [ ]:
#scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [ ]:
#build model
def build_model(n_hidden = 1, n_neurons = 30, learning_rate = 3e-3, input_shape = X_train.shape[1:]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape = input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation = "relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss = "mse", optimizer=optimizer)
    return model

In [ ]:
#create a KerasRegressor
#keras_reg = scikeras.wrappers.KerasRegressor(build_model)
keras_reg = scikeras.wrappers.KerasRegressor(
    build_model,
    n_neurons=91,
    n_hidden=3,
    optimizer__lr=3e-3,  # just specify parameters here, you won't have to edit build_model to add/remove things you're tuning
    optimizer=keras.optimizers.SGD,  # note that I am giving SciKeras a class, not an instance
   loss="mse",
)

In [ ]:
keras_reg.fit(X_train, y_train,epochs = 100, validation_data = (X_valid,y_valid), callbacks = [keras.callbacks.EarlyStopping(patience=10)])
mse_test = keras_reg.score(X_test, y_test)
X_new = X_test
y_pred = keras_reg.predict(X_new)

In [ ]:
print(y_pred)
print(y_test)

In [ ]:
#explore
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [0, 1, 2, 3, 4, 5],
    #"n_neurons": np.arange(1, 100),
    "n_neurons" : np.arange(1, 100, 10),
    #"optimizer__lr": reciprocal(3e-4, 3e-2),
    "optimizer__lr" : np.arange(3e-4, 3e-2, 0.3*(1e-2 - 1e-4)),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)
rnd_search_cv.fit(
    X_train,
    y_train,
    epochs=25,
    validation_data=(X_valid, y_valid),
    callbacks=[keras.callbacks.EarlyStopping(patience=10)],
)

In [ ]:
rnd_search_cv.best_params_

In [ ]:
#{'optimizer__lr': 0.01218, 'n_neurons': 91, 'n_hidden': 3}
#{'optimizer__lr': 0.01515, 'n_neurons': 61, 'n_hidden': 5}